In [1]:
import numpy as np
import pandas as pd
from concurrent.futures import ThreadPoolExecutor
import requests

In [2]:
links = pd.read_csv('./links.csv')[['movieId', 'imdbId']]

In [3]:
movie_json = [None]*len(links)

In [4]:
def fetch_movie(data, index):
    imdb_id = data
    imdb_id = str(imdb_id)
    imdb_id = 'tt'+'0'*(7-len(imdb_id)) + imdb_id
    API_KEY = "62c56bbac67642762ecb788540b5888e"
    url = f"https://api.themoviedb.org/3/find/{imdb_id}?api_key={API_KEY}&external_source=imdb_id"
    try: 
        tmdb_id = requests.get(url).json()['movie_results'][0]['id']
        url=  f"http://api.themoviedb.org/3/movie/{tmdb_id}?api_key={API_KEY}&append_to_response=videos"
        # url = f"https://api.themoviedb.org/3/movie/{tmdb_id}?api_key={API_KEY}"
        r = requests.get(url).json()
        movie_json[index] = r
    except: 
        movie_json[index] = None

In [5]:
with ThreadPoolExecutor(max_workers=100) as executor:
    with requests.Session() as session:
        executor.map(fetch_movie, links['imdbId'].tolist(), list(range(len(links))))
        executor.shutdown(wait=True)

In [6]:
pd.DataFrame(movie_json).to_csv('./movie_json.csv')

In [7]:
movie_json = pd.read_csv('./movie_json.csv')

In [8]:
movie_json.head()

,Unnamed: 0,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,videos
0,0,False,/yZK4T7ZDQcQr5omvSnJnR9IzBVa.jpg,"{'id': 2602, 'name': 'Scream Collection', 'pos...",40000000,"[{'id': 27, 'name': 'Horror'}, {'id': 9648, 'n...",NaN,4234,tt0134084,en,...,161834276,116,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Someone has taken their love of trilogies one ...,Scream 3,False,5.9,2542,"{'results': [{'iso_639_1': 'en', 'iso_3166_1':..."
1,1,False,/105h2E4rDX65Tzs4bWrR4rYhElU.jpg,"{'id': 87186, 'name': 'The Boondock Saints Col...",6000000,"[{'id': 28, 'name': 'Action'}, {'id': 53, 'nam...",http://www.theboondocksaints.com,8374,tt0144117,en,...,50000000,108,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Thy Kingdom Come. Thy Will Be Done.,The Boondock Saints,False,7.3,1923,"{'results': [{'iso_639_1': 'en', 'iso_3166_1':..."
2,2,False,/cxRcmEwtUZVfNlC4HoWaNKzNNW0.jpg,NaN,40000000,"[{'id': 18, 'name': 'Drama'}, {'id': 12, 'name...",NaN,1907,tt0163978,en,...,144056873,119,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Somewhere on this planet it must exist.,The Beach,False,6.5,3638,"{'results': [{'iso_639_1': 'en', 'iso_3166_1':..."
3,3,False,/3xIzrSM5GKoF8JAlUnnSaG04gCS.jpg,NaN,7000000,"[{'id': 80, 'name': 'Crime'}, {'id': 18, 'name...",NaN,14181,tt0181984,en,...,28780255,118,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Welcome to the new American dream.,Boiler Room,False,6.7,513,"{'results': [{'iso_639_1': 'en', 'iso_3166_1':..."
4,4,False,/jIibfKqDN5i5ZU0vshk06UEUaVg.jpg,"{'id': 2794, 'name': 'The Chronicles of Riddic...",23000000,"[{'id': 53, 'name': 'Thriller'}, {'id': 878, '...",http://www.pitchblack.com/,2787,tt0134847,en,...,53187659,108,"[{'english_name': 'Arabic', 'iso_639_1': 'ar',...",Released,Don't be afraid of the dark. Be afraid of what...,Pitch Black,False,6.8,3659,"{'results': [{'iso_639_1': 'en', 'iso_3166_1':..."


In [9]:
movie_json.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048 entries, 0 to 1047
Data columns (total 27 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             1048 non-null   int64  
 1   adult                  1048 non-null   bool   
 2   backdrop_path          1048 non-null   object 
 3   belongs_to_collection  359 non-null    object 
 4   budget                 1048 non-null   int64  
 5   genres                 1048 non-null   object 
 6   homepage               581 non-null    object 
 7   id                     1048 non-null   int64  
 8   imdb_id                1048 non-null   object 
 9   original_language      1048 non-null   object 
 10  original_title         1048 non-null   object 
 11  overview               1048 non-null   object 
 12  popularity             1048 non-null   float64
 13  poster_path            1048 non-null   object 
 14  production_companies   1048 non-null   object 
 15  prod

In [10]:
df = movie_json[['backdrop_path', 'genres', 'imdb_id', 'original_title', 'overview', 'vote_count', 'vote_average', 'videos']]

In [11]:
df['movieId'] = links['movieId'][:]

/var/folders/t7/y348wtp93rv55cxh2hghzhqc0000gn/T/ipykernel_4236/19923144.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['movieId'] = links['movieId'][:]


In [12]:
import json
df['genres'] = df['genres'].apply(lambda x: json.loads(x.replace("'", '"')))

/var/folders/t7/y348wtp93rv55cxh2hghzhqc0000gn/T/ipykernel_4236/131765160.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['genres'] = df['genres'].apply(lambda x: json.loads(x.replace("'", '"')))


In [13]:
df['genres'] = df['genres'].apply(lambda x: [y['name'] for y in x])

/var/folders/t7/y348wtp93rv55cxh2hghzhqc0000gn/T/ipykernel_4236/980155116.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['genres'] = df['genres'].apply(lambda x: [y['name'] for y in x])


In [14]:
df.to_csv('./movie_info.csv')

In [20]:
pd.read_csv('./movie_info.csv', index_col='movieId').head(3)

,Unnamed: 0,backdrop_path,genres,imdb_id,original_title,overview,vote_count,vote_average,videos
movieId,,,,,,,,,
3273,0,/yZK4T7ZDQcQr5omvSnJnR9IzBVa.jpg,"['Horror', 'Mystery']",tt0134084,Scream 3,As bodies begin dropping around the set of ‘St...,2542,5.9,"{'results': [{'iso_639_1': 'en', 'iso_3166_1':..."
3275,1,/105h2E4rDX65Tzs4bWrR4rYhElU.jpg,"['Action', 'Thriller', 'Crime']",tt0144117,The Boondock Saints,Tired of the crime overrunning the streets of ...,1923,7.3,"{'results': [{'iso_639_1': 'en', 'iso_3166_1':..."
3285,2,/cxRcmEwtUZVfNlC4HoWaNKzNNW0.jpg,"['Drama', 'Adventure', 'Romance', 'Thriller']",tt0163978,The Beach,Twenty-something Richard travels to Thailand a...,3638,6.5,"{'results': [{'iso_639_1': 'en', 'iso_3166_1':..."


In [16]:
import ast
for i in range(1048):
    try: 
        df['videos'].iloc[i] = ast.literal_eval(df['videos'].iloc[i])
    except: 
        print(i)
        print(df['videos'][i])
        break; 

/var/folders/t7/y348wtp93rv55cxh2hghzhqc0000gn/T/ipykernel_4236/2743779768.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['videos'].iloc[i] = ast.literal_eval(df['videos'].iloc[i])
/var/folders/t7/y348wtp93rv55cxh2hghzhqc0000gn/T/ipykernel_4236/2743779768.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['videos'].iloc[i] = ast.literal_eval(df['videos'].iloc[i])
/var/folders/t7/y348wtp93rv55cxh2hghzhqc0000gn/T/ipykernel_4236/2743779768.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs

In [17]:
df['videos'].iloc[0]['results']

[{'iso_639_1': 'en',
  'iso_3166_1': 'US',
  'name': 'Scream 3 (2000) - Movie Trailer',
  'key': 'bYi-rmHfrP8',
  'published_at': '2010-12-02T21:55:13.000Z',
  'site': 'YouTube',
  'size': 360,
  'type': 'Trailer',
  'official': False,
  'id': '533ec65ec3a3685448000e62'}]

In [ ]:
df.to_csv('./movie_info.csv', index=None)